In [33]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
from scipy.stats import norm
import copy
import os
import plotly.express as px

from sklearn.linear_model import LinearRegression


# Plotly imports
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
plotly.offline.init_notebook_mode(connected=True)

from itertools import chain, product

#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data


In [34]:
#Stock Index of Interest
stock_ident = "BANKNIFTY"
# stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "ATM"

#Product type to hedge: either "CE" or "PE"
# prod_type = "PE"
prod_type = "CE"

# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path


# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020:[1,2,3,4,5,6,7]}
# periods_of_interest = {2020:[2,3,4,5]}
#August - Good Vol smile


#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]

#Simulation Parameters
no_of_paths = 5000

#Number of options
no_opt=1

outlier_ind = 0


In [38]:
#####################################
# Input the final resutls dataframe #
#####################################
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
df_hedge = pd.read_csv(output_data_path + "E4A_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_hedge_results.csv")
covid_period = 0
# mth_wise_ind = 1
mth_wise_ind = 999
if(covid_period == 1):  
    month_list = [date(2020, 2, 27), date(2020, 3, 26), date(2020, 4, 30)]
    mth_list = [monthly_dates.strftime("%d-%b-%Y") for monthly_dates in month_list]
    df_hedge = df_hedge[df_hedge.Month_Expiry.isin(mth_list)]
if (mth_wise_ind != 999):
        df_hedge = df_hedge[df_hedge.Month_Expiry.isin([mthly_expiries_list[mth_wise_ind.strftime("%d-%b-%Y")]])]

print(df_hedge.columns)   

Index(['Date', 'Month_Expiry', 'Underlying Value', 'Underlying Value - Lag',
       'Date_lag', 'Strike Price', 'r', 'expiry_dt', 'Close', 'Close_lag',
       'Pfl PnL - Realised', 'iv', 'delta', 'delta_lag',
       'Hedge Pfl Value - Dynamic', 'dynamic_pfl_lag', 'Pfl PnL - Dynamic',
       'Week_Start', 'Week_Expiry', 'Hedge Pfl Value - ConstVol',
       'Static Hedge Error - ConstVol', 'Pfl PnL - ConstVol',
       'Hedge Pfl Value - ConstVolSmile', 'Static Hedge Error - ConstVolSmile',
       'Pfl PnL - ConstVolSmile', 'Hedge Pfl Value - ConstVolSurf',
       'Static Hedge Error - ConstVolSurf', 'Pfl PnL - ConstVolSurf',
       'Hedge Pfl Value - FwdSmile', 'Static Hedge Error - FwdSmile',
       'Pfl PnL - FwdSmile'],
      dtype='object')


In [39]:
# Scatter PLot with Regression: Realised PnL vs Model PnL
# Line plot of PnL Errors w.r.to Realised PnL
    # PLot Dynamic PnL Error vs 4 model errors.



# Create inputs for plotting function

#Input:
#Datafrmae of interest - pandas.dataframe
# outlier_treatment = 0 if outliers are not removed. 
# Outlier treatment = 1 if outliers are remved

def generate_plot_inputs_staticHedge_PV_regression(df_hedge, no_of_plots, \
                                                    prod_type, stock_ident, prod_moneyness, \
                                                    outlier_treatment = 0):
    if (outlier_treatment == 1):
        df_hedge['Outlier_diff'] = df_hedge['Close'] - df_hedge['Hedge Pfl Value - ConstVol'] 
        
        perctls_dyn = [np.percentile(np.array(df_hedge['Outlier_diff']), 2), \
                       np.percentile(np.array(df_hedge['Outlier_diff']), 98)]
        df_hedge = df_hedge[df_hedge['Outlier_diff'] > perctls_dyn[0]]
        df_hedge = df_hedge[df_hedge['Outlier_diff'] < perctls_dyn[1]]
    
    
    realised_pnL = df_hedge['Close']
    model_pnLs = ['Hedge Pfl Value - ConstVol', 'Hedge Pfl Value - ConstVolSmile', \
                 'Hedge Pfl Value - ConstVolSurf', 'Hedge Pfl Value - FwdSmile']

    x_list = [np.array(realised_pnL).reshape(-1) for plot in range(0, no_of_plots)]
    y_list = [[np.array(df_hedge[model_pnLs[plot]]).reshape(-1)] for plot in range(0, no_of_plots)]
    
    #Titles
    if (prod_type == "CE"):
        option_type = "Call Option"
    else:
        option_type = "Put Option"
        
    figure_title_str = stock_ident + " - Actual PV vs Static Hedge PV - " + prod_moneyness + " " + option_type
    
    #Sub-title names
    subplot_title_str_list = ['Constant Volatility', 'Constant Volatility Smile', \
                         'Constant Volatility Surface', 'Forward Smile']
    #Legends
    legends_str = [[subplot_title_str_list[plot], "Regression Fit"] for plot in range(0, no_of_plots)]
    
    #Axes titles
    xaxis_title_str = ["Realised PV" for plot in range(0, no_of_plots)]
    yaxis_title_str = ["Model  PV" for plot in range(0, no_of_plots)]
    
    for i in range(0, no_of_plots):
        if (i==0):
            xaxis_title_str[i] = ""
        elif (i==1):
            xaxis_title_str[i] = ""
            yaxis_title_str[i] = ""
        elif(i==3):
            yaxis_title_str[i] = ""
        else:
            None
    
    #Limits
    x_limits = [[np.amin(x_list[plot])-100, np.amax(x_list[plot])+100] for plot in range(0,no_of_plots)]
    y_limits = [[np.amin(y_list[plot])-100, np.amax(y_list[plot])+100] for plot in range(0,no_of_plots)]
    
    
    # Annotations
    lin_reg_fits = [LinearRegression(fit_intercept=False).fit(np.array(x_list[plot]).reshape(-1,1), \
                                                             np.array(y_list[plot][0]).reshape(-1,1)) for plot in range(0, no_of_plots) ] 
    
    other_annotations = ["R-Square: " + str(round(lin_reg_fits[plot].score(np.array(x_list[plot]).reshape(-1,1), \
                                                  np.array(y_list[plot][0]).reshape(-1,1))*100,2)) + "%" \
                          + "<br>" \
                          "Model PV = " + str(round(lin_reg_fits[plot].coef_[0][0], 2)) + " * Realised PV" \
                         for plot in range(0, no_of_plots) ]
    
    regression_coefficients = [lin_reg_fits[plot].coef_[0][0] for plot in range(0, no_of_plots)]
    y_reg_line = [np.array([regression_coefficients[plot] * x_list[plot]]).reshape(-1) for plot in range(0, no_of_plots)]
    y_list = [[y_list[plot][0], y_reg_line[plot]] for plot in range(0, no_of_plots)] 

    return figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, \
           legends_str, other_annotations, x_limits, y_limits, x_list, y_list



# Create inputs for plotting function

#Input:
#Datafrmae of interest - pandas.dataframe
# outlier_treatment = 0 if outliers are not removed. 
# Outlier treatment = 1 if outliers are remved

def generate_plot_inputs_dynamicHedge_PV_regression(df_hedge, no_of_plots, \
                                                    prod_type, stock_ident, prod_moneyness, \
                                                    outlier_treatment = 0):
    if (outlier_treatment == 1):
        df_hedge['Outlier_diff'] = df_hedge['Close'] - df_hedge['Hedge Pfl Value - Dynamic'] 
        
        perctls_dyn = [np.percentile(np.array(df_hedge['Outlier_diff']), 1), \
                       np.percentile(np.array(df_hedge['Outlier_diff']), 98)]
        df_hedge = df_hedge[df_hedge['Outlier_diff'] > perctls_dyn[0]]
        df_hedge = df_hedge[df_hedge['Outlier_diff'] < perctls_dyn[1]]
        
    
    realised_pnL = df_hedge['Close']
    model_pnLs = ['Hedge Pfl Value - Dynamic', 'Hedge Pfl Value - ConstVol']
    dynamic_pnL = df_hedge[model_pnLs[0]]
    x_list = [np.array(realised_pnL).reshape(-1) for plot in range(0, no_of_plots)]
    y_list = [[np.array(df_hedge[model_pnLs[plot]]).reshape(-1)] for plot in range(0, no_of_plots)]
    
    #Titles
    if (prod_type == "CE"):
        option_type = "Call Option"
    else:
        option_type = "Put Option"
        
    figure_title_str = stock_ident + " - Realised PV vs Static/Dynamic Hedge PV - " + prod_moneyness + " " + option_type
    
    #Sub-title names
    subplot_title_str_list = ['Dynamic Hedge', 'Static Hedge']
    #Legends
    legends_str = [[subplot_title_str_list[plot], "Regression Fit"] for plot in range(0, no_of_plots)]
    
    #Axes titles
    xaxis_title_str = ["Realised PV" for plot in range(0, no_of_plots)]
    yaxis_title_str = ["Model  PV" for plot in range(0, no_of_plots)]
    
    for i in range(0, no_of_plots):
        if (i==1):
            yaxis_title_str[i] = ""
        else:
            None
    
    #Limits
    x_limits = [[np.amin(x_list[plot]) -100, np.amax(x_list[plot]) + 100] for plot in range(0,no_of_plots)]
    y_limits = [[np.amin(y_list[plot]) -100, np.amax(y_list[plot])+100] for plot in range(0,no_of_plots)]
    
    
    # Annotations
    lin_reg_fits = [LinearRegression(fit_intercept=False).fit(np.array(x_list[plot]).reshape(-1,1), \
                                                             np.array(y_list[plot][0]).reshape(-1,1)) for plot in range(0, no_of_plots) ] 
    
    other_annotations = ["R-Square: " + str(round(lin_reg_fits[plot].score(np.array(x_list[plot]).reshape(-1,1), \
                                                  np.array(y_list[plot][0]).reshape(-1,1))*100,2)) + "%" \
                          + "<br>" \
                          "Model PV = " + str(round(lin_reg_fits[plot].coef_[0][0], 2)) + " * Realised PV" \
                         for plot in range(0, no_of_plots) ]
    
    regression_coefficients = [lin_reg_fits[plot].coef_[0][0] for plot in range(0, no_of_plots)]
    y_reg_line = [np.array([regression_coefficients[plot] * x_list[plot]]).reshape(-1) for plot in range(0, no_of_plots)]
    y_list = [[y_list[plot][0], y_reg_line[plot]] for plot in range(0, no_of_plots)] 

    return figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, \
           legends_str, other_annotations, x_limits, y_limits, x_list, y_list


    
# Inputs

# 1. no_of_plots: int ----> No. of. plots in the Figure
# 2. no_of_rows: int -----> No. of. rows in the Figure
# 3. no_of_cols: int. -----> No. of. cols in the Figure
# 4. no_of_y_vars: int -----> No. of y-variables in each plot for a single x-axis
# 5. figure_title_str: str ----> Figure title
# 6. subplot_title_str_list: str ----> Figure Title
# 7. xaxis_title_str: list: -----> x_axis title string for each plot
# 8. yaxis_title_strL list: -----> y-axis title string for each plot
# 9. legends_str: list ----> [[y1_legend, y2_legend,...], [y1_legend, y2_legend,..], [],..] 
                             # ---> list (of length 'no_of_plots') of list of y_axes legends 
                             #       for each y-axis variable 
# 10. x_list: list -----> List of x axis numpy arrays for each sub-plot
# 11. y_lsit: list ------> Contains no.of.sublots length of list of list of numpy arrays of length no_of_yvars
# 12. regression_indicator: Int: 0 for no regression line, 1 for regression line.

def generate_plots(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, \
                  figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, \
                  other_annotations):
    
    fig = make_subplots(rows=no_of_rows, cols=no_of_cols, subplot_titles=subplot_title_str_list)
    fig.update_layout(height=700, width=900, \
                     title=dict(text=figure_title_str, y=0.95, x=0.5, xanchor='center', yanchor='top'), \
                     font=dict(color="black", size=14), showlegend=False)
    figure_positions = list(product(range(1, no_of_rows+1), range(1, no_of_cols+1)))
    colours_list = ['rgb(57,105,172)', 'rgb(207, 28, 144)', 'rgb(17, 165, 121)', 'rgb(230, 131, 16)', 'rgb(139, 0, 0)', 'rgb(0, 0, 139)', 'rgb(0, 139, 0)']
    legend_show_list = [False] + [False for i in range(0, no_of_plots-1)]
    
    for i in range(0, no_of_plots):
        for j in range(0, no_of_yvars):
            if (j==0):
                plt_color = colours_list[i]
                plt_mode='markers'
            else:
                plt_color = 'rgb(153, 153, 153)'
                plt_mode='lines'
            
            fig.add_trace(go.Scatter(x=list(x_list[i]), y=list(y_list[i][j]), \
                                     mode=plt_mode, marker=dict(size=4, color=plt_color), \
                                     name=legends_str[i][j], showlegend=legend_show_list[i]), \
                          row=figure_positions[i][0], col=figure_positions[i][1])
    
    for i in range(0, no_of_plots):
        fig.update_xaxes(title_text=xaxis_title_str[i], showgrid=True, range=x_limits[i], \
                        row=figure_positions[i][0], col=figure_positions[i][1])
        fig.update_yaxes(title_text=yaxis_title_str[i], showgrid=True, range=y_limits[i], \
                        row=figure_positions[i][0], col=figure_positions[i][1])
    
    annotations=[]
    annotations_extend = [dict(x=0.01, y=0.999, xref="paper", yref="paper", \
                              text=other_annotations[0], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                          dict(x=0.56, y=0.999, xref="paper", yref="paper", \
                              text=other_annotations[1], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                         dict(x=0.01, y=0.37, xref="paper", yref="paper", \
                              text=other_annotations[2], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                         dict(x=0.56, y=0.37, xref="paper", yref="paper", \
                              text=other_annotations[3], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                         ]
    annotations = [annotation for annotation in fig["layout"]["annotations"]]
    annotations = annotations + annotations_extend
    fig["layout"]["annotations"] += tuple(annotations)
    
    return fig

def generate_plots_dynamic(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, \
                  figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, \
                  other_annotations):
    
    fig = make_subplots(rows=no_of_rows, cols=no_of_cols, subplot_titles=subplot_title_str_list)
    fig.update_layout(height=400, width=900, \
                     title=dict(text=figure_title_str, y=0.95, x=0.5, xanchor='center', yanchor='top'), \
                     font=dict(color="black", size=14), showlegend=False)
    figure_positions = list(product(range(1, no_of_rows+1), range(1, no_of_cols+1)))
    colours_list = ['rgb(152, 78,163)', 'rgb(57,105,172)', 'rgb(231, 138,195)', 'rgb(57,105,172)', 'rgb(207, 28, 144)', 'rgb(17, 165, 121)', 'rgb(230, 131, 16)', 'rgb(139, 0, 0)', 'rgb(0, 0, 139)', 'rgb(0, 139, 0)']
    legend_show_list = [False] + [False for i in range(0, no_of_plots-1)]
    
    for i in range(0, no_of_plots):
        for j in range(0, no_of_yvars):
            if (j==0):
                plt_color = colours_list[i]
                plt_mode='markers'
            else:
                plt_color = 'rgb(153, 153, 153)'
                plt_mode='lines'
            
            fig.add_trace(go.Scatter(x=list(x_list[i]), y=list(y_list[i][j]), \
                                     mode=plt_mode, marker=dict(size=4, color=plt_color), \
                                     name=legends_str[i][j], showlegend=legend_show_list[i]), \
                          row=figure_positions[i][0], col=figure_positions[i][1])
    
    for i in range(0, no_of_plots):
        fig.update_xaxes(title_text=xaxis_title_str[i], showgrid=True, range=x_limits[i], \
                        row=figure_positions[i][0], col=figure_positions[i][1])
        fig.update_yaxes(title_text=yaxis_title_str[i], showgrid=True, range=y_limits[i], \
                        row=figure_positions[i][0], col=figure_positions[i][1])
    
    annotations=[]
    annotations_extend = [dict(x=0.22, y=0.25, xref="paper", yref="paper", \
                              text=other_annotations[0], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=10)), \
                          dict(x=0.77, y=0.25, xref="paper", yref="paper", \
                              text=other_annotations[1], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                         ]
    annotations = [annotation for annotation in fig["layout"]["annotations"]]
    annotations = annotations + annotations_extend
    fig["layout"]["annotations"] += tuple(annotations)
    
    return fig



In [40]:
# Regression plots for Static Hedging and Dynamic Hedging

no_of_plots=4
no_of_rows = 2
no_of_cols = 2
no_of_yvars = 2
outlier_treatment = 0
figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, \
legends_str, other_annotations, x_limits, y_limits, x_list, y_list = generate_plot_inputs_staticHedge_PV_regression(df_hedge, no_of_plots, \
                                                    prod_type, stock_ident, prod_moneyness, \
                                                    outlier_treatment)

reg_static_hedge_plot = generate_plots(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, \
                          figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, \
                          other_annotations)
reg_static_hedge_plot.show()

if (outlier_treatment == 0):
    reg_static_hedge_plot.write_image(output_data_path + "P2_"+ stock_ident + "_" + prod_moneyness + "_" + prod_type  \
                         +'_static_regression_fit.png')
else:
    reg_static_hedge_plot.write_image(output_data_path + "P2_"+ stock_ident + "_" + prod_moneyness + "_" + prod_type  \
                         +'_static_regression_fit_outlier_treated.png')    

no_of_plots=2
no_of_rows = 1
no_of_cols = 2
no_of_yvars = 2
outlier_treatment = 0

figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, \
legends_str, other_annotations, x_limits, y_limits, x_list, y_list = generate_plot_inputs_dynamicHedge_PV_regression(df_hedge, no_of_plots, \
                                                    prod_type, stock_ident, prod_moneyness, \
                                                    outlier_treatment)

reg_dynamic_hedge_plot = generate_plots_dynamic(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, \
                          figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, \
                          other_annotations)
reg_dynamic_hedge_plot.show()

# if (outlier_treatment == 0):
#     reg_dynamic_hedge_plot.write_image(output_data_path + "P2_"+ stock_ident + "_" + prod_moneyness + "_" + prod_type  \
#                                        + '_dynamic_regression_fit.png')
# else:
#     reg_dynamic_hedge_plot.write_image(output_data_path + "P2_"+ stock_ident + "_" + prod_moneyness + "_" + prod_type  \
#                                        + '_dynamic_regression_fit_outlier_treated.png')    



In [41]:
# PnL Error Plots

def generate_plot_inputs_PV_Errors(df_hedge, no_of_plots, \
                                                    prod_type, stock_ident, prod_moneyness, \
                                                    outlier_treatment = 0):

    model_pnLs = ['Hedge Pfl Value - Dynamic', 'Hedge Pfl Value - ConstVol', 'Hedge Pfl Value - ConstVolSmile', \
                 'Hedge Pfl Value - ConstVolSurf', 'Hedge Pfl Value - FwdSmile']
    
    df_hedge['PnL Diff: Dynamic Hedge'] = df_hedge['Close'] - df_hedge['Hedge Pfl Value - Dynamic']
    df_hedge['PnL Diff: Constant Vol Static Hedge'] = df_hedge['Close'] - df_hedge['Hedge Pfl Value - ConstVol']
    df_hedge['PnL Diff: Constant Vol Smile Static Hedge'] = df_hedge['Close'] - df_hedge['Hedge Pfl Value - ConstVolSmile']
    df_hedge['PnL Diff: Constant Vol Surface Static Hedge'] = df_hedge['Close'] - df_hedge['Hedge Pfl Value - ConstVolSurf']
    df_hedge['PnL Diff: Forward Smile Static Hedge'] = df_hedge['Close'] - df_hedge['Hedge Pfl Value - FwdSmile']
    
    
    if (outlier_treatment == 1):
        dyn_abs_error_pctl = np.percentile(np.absolute(np.array(df_hedge['PnL Diff: Dynamic Hedge'])),99)
        consVol_abs_error_pctl = np.percentile(np.absolute(np.array(df_hedge['PnL Diff: Constant Vol Static Hedge'])),99)
        consVolSmile_abs_error_pctl = np.percentile(np.absolute(np.array(df_hedge['PnL Diff: Constant Vol Smile Static Hedge'])),99)
        consVolSurf_abs_error_pctl = np.percentile(np.absolute(np.array(df_hedge['PnL Diff: Constant Vol Surface Static Hedge'])),99)
        fwdVol_abs_error_pctl = np.percentile(np.absolute(np.array(df_hedge['PnL Diff: Forward Smile Static Hedge'])),99)
        abs_pctl = np.amin(np.array([dyn_abs_error_pctl, consVol_abs_error_pctl, \
                                     consVolSmile_abs_error_pctl, consVolSurf_abs_error_pctl, 
                                     fwdVol_abs_error_pctl]))
        
        df_hedge['PnL Abs Diff: Dynamic Hedge'] = np.absolute(np.array(df_hedge['PnL Diff: Dynamic Hedge']))
        df_hedge['PnL Abs Diff: Constant Vol Static Hedge'] = np.absolute(np.array(df_hedge['PnL Diff: Constant Vol Static Hedge']))
        df_hedge['PnL Abs Diff: Constant Vol Smile Static Hedge'] = np.absolute(np.array(df_hedge['PnL Diff: Constant Vol Smile Static Hedge']))
        df_hedge['PnL Abs Diff: Constant Vol Surface Static Hedge'] = np.absolute(np.array(df_hedge['PnL Diff: Constant Vol Surface Static Hedge']))
        df_hedge['PnL Abs Diff: Forward Smile Static Hedge'] = np.absolute(np.array(df_hedge['PnL Diff: Forward Smile Static Hedge']))
        
        df_hedge = df_hedge[df_hedge['PnL Abs Diff: Dynamic Hedge'] < abs_pctl]
        df_hedge = df_hedge[df_hedge['PnL Abs Diff: Constant Vol Static Hedge'] < abs_pctl]
        df_hedge = df_hedge[df_hedge['PnL Abs Diff: Constant Vol Smile Static Hedge'] < abs_pctl]
        df_hedge = df_hedge[df_hedge['PnL Abs Diff: Constant Vol Surface Static Hedge'] < abs_pctl]
        df_hedge = df_hedge[df_hedge['PnL Abs Diff: Forward Smile Static Hedge'] < abs_pctl]
#         print(np.amax(np.array(df_hedge['PnL Diff: Dynamic Hedge'])))
               
    pnL_diffs_names = ['PnL Diff: Dynamic Hedge', 'PnL Diff: Constant Vol Static Hedge', \
                      'PnL Diff: Constant Vol Smile Static Hedge', 'PnL Diff: Constant Vol Surface Static Hedge', \
                      'PnL Diff: Forward Smile Static Hedge']
    
    
    date_list = np.array(df_hedge['Date']).reshape(-1)
    get_date = lambda each_date: datetime.strptime(each_date, "%d-%b-%Y").date()
    get_date_vector = np.vectorize(get_date)
    
    date_list = get_date_vector(date_list)
    
    x_list = [date_list for plot in range(0, no_of_plots)]
#     y_list = [[np.absolute(np.array(df_hedge[pnL_diffs_names[0]])).reshape(-1), \
#               np.absolute(np.array(df_hedge[pnL_diffs_names[plot]])).reshape(-1)] for plot in range(1, no_of_plots+1)]
    y_list = [[np.array(df_hedge[pnL_diffs_names[0]]).reshape(-1), \
              np.array(df_hedge[pnL_diffs_names[plot]]).reshape(-1)] for plot in range(1, no_of_plots+1)]

    #Titles
    if (prod_type == "CE"):
        option_type = "Call Option"
    else:
        option_type = "Put Option"
        
    figure_title_str = stock_ident + ": PV Errors - Static vs Dynamic Hedge - " + prod_moneyness + " " + option_type
    
    #Sub-title names
    subplot_title_str_list = ['Constant Volatility vs Dynamic', 'Constant Volatility Smile vs Dynamic', \
                              'Constant Volatility Surface vs Dynamic', 'Forward Smile vs Dynamic']
    #Legends
    legend_list = ['Constant Volatility', 'Constant Volatility Smile', \
                   'Constant Volatility Surface', 'Forward Smile']
    legends_str = [["Dynamic Hedge", legend_list[plot]] for plot in range(0, no_of_plots)]
    
    for i in range(0, no_of_plots):
        if (i!=0):
            legends_str[i][0] = ""
    
    #Axes titles
    xaxis_title_str = ["Date" for plot in range(0, no_of_plots)]
    yaxis_title_str = ["PV Difference" for plot in range(0, no_of_plots)]
    
    for i in range(0, no_of_plots):
        if (i==0):
            xaxis_title_str[i] = ""
        elif (i==1):
            xaxis_title_str[i] = ""
            yaxis_title_str[i] = ""
        elif(i==3):
            yaxis_title_str[i] = ""
        else:
            None
    
    #Limits
    x_limits = [[np.amin(x_list[plot]) - timedelta(days=7), np.amax(x_list[plot]) + timedelta(days=7)] for plot in range(0,no_of_plots)]
    
    y_limits = [[np.amin(y_list[plot]) -5, np.amax(y_list[plot]) + 10] for plot in range(0,no_of_plots)]
    
        
    other_annotations = ["Mean Abs diff (Dynamic): " + str(round(np.mean(np.absolute(y_list[plot][0])), 2)) \
                          + "<br>" \
                          "Mean Abs diff (Static) = " + str(round(np.mean(np.absolute(y_list[plot][1])), 2)) \
                         for plot in range(0, no_of_plots)]
    return figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, \
           legends_str, other_annotations, x_limits, y_limits, x_list, y_list


def generate_pv_error_plots(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, \
                  figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, \
                  other_annotations):
    
    fig = make_subplots(rows=no_of_rows, cols=no_of_cols, subplot_titles=subplot_title_str_list)
    fig.update_layout(height=800, width=1000, \
                     title=dict(text=figure_title_str, y=0.95, x=0.42, xanchor='center', yanchor='top'), \
                     font=dict(color="black", size=14), showlegend=True)
    figure_positions = list(product(range(1, no_of_rows+1), range(1, no_of_cols+1)))
    colours_list = ['rgb(57,105,172)', 'rgb(217,95,2)', 'rgb(207, 28, 144)', 'rgb(17, 165, 121)', 'rgb(230, 131, 16)', 'rgb(57,105,172)', \
                    'rgb(93,105,177)', 'rgb(57,105,172)', 'rgb(139, 0, 0)', 'rgb(0, 0, 139)', 'rgb(0, 139, 0)']
#     legend_show_list = [True] + [True for i in range(0, no_of_plots-1)]
    
    for i in range(0, no_of_plots):
        for j in range(0, no_of_yvars):
            if (j==1):
                plt_color = colours_list[i]
                plt_mode='lines+markers'
                legend_show=True
            else:
                plt_color = 'rgb(102, 102, 102)'
                plt_mode='lines+markers'
                if(i==0):
                    legend_show=True
                else:
                    legend_show=False
                      
            fig.add_trace(go.Scatter(x=list(x_list[i]), y=list(y_list[i][j]), \
                                     mode=plt_mode, marker=dict(size=4, color=plt_color), \
                                     name=legends_str[i][j], showlegend=legend_show), \
                          row=figure_positions[i][0], col=figure_positions[i][1])
    
    for i in range(0, no_of_plots):
        fig.update_xaxes(title_text=xaxis_title_str[i], showgrid=True, range=x_limits[i], \
                        row=figure_positions[i][0], col=figure_positions[i][1])
        fig.update_yaxes(title_text=yaxis_title_str[i], showgrid=True, range=y_limits[i], \
                        row=figure_positions[i][0], col=figure_positions[i][1])
    
    annotations=[]
    annotations_extend = [dict(x=0.01, y=0.999, xref="paper", yref="paper", \
                              text=other_annotations[0], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                          dict(x=0.56, y=0.999, xref="paper", yref="paper", \
                              text=other_annotations[1], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                         dict(x=0.01, y=0.37, xref="paper", yref="paper", \
                              text=other_annotations[2], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                         dict(x=0.56, y=0.37, xref="paper", yref="paper", \
                              text=other_annotations[3], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                         ]
    annotations = [annotation for annotation in fig["layout"]["annotations"]]
    annotations = annotations + annotations_extend
    fig["layout"]["annotations"] += tuple(annotations)
    
    return fig


# Regression plots for Static Hedging and Dynamic Hedging

no_of_plots=4
no_of_rows = 2
no_of_cols = 2
no_of_yvars = 2
outlier_treatment = 0
figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, \
legends_str, other_annotations, x_limits, y_limits, x_list, y_list = generate_plot_inputs_PV_Errors(df_hedge, no_of_plots, \
                                                    prod_type, stock_ident, prod_moneyness, \
                                                    outlier_treatment)

reg_static_hedge_pnLerror_plot = generate_pv_error_plots(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, \
                          figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, \
                          other_annotations)
reg_static_hedge_pnLerror_plot.show()

# if (outlier_treatment == 0):
#     reg_static_hedge_pnLerror_plot.write_image(output_data_path + "P2_"+ stock_ident + "_" + prod_moneyness + "_" + prod_type  \
#                          +'_static_vs_dynamic_pnL_error.png')
# else:
#     reg_static_hedge_pnLerror_plot.write_image(output_data_path + "P2_"+ stock_ident + "_" + prod_moneyness + "_" + prod_type  \
#                          +'_static_vs_dynamic_pnL_error.png')   




In [51]:

# Scatter PLot with Regression: Realised PnL vs Model PnL
# Line plot of PnL Errors w.r.to Realised PnL
    # PLot Dynamic PnL Error vs 4 model errors.

#####################################
# Input the final resutls dataframe #
#####################################
prod_type = "CE"

# df_hedge_carrWu = pd.read_csv(output_data_path + "E0B_" + stock_ident + "_" + prod_moneyness + "_" + "CE" + "_hedge_results.csv")
df_hedge_carrWu = pd.read_csv(output_data_path + "E0C_" + stock_ident + "_" + prod_moneyness + "_" + "CE" + "_hedge_results.csv")


# Create inputs for plotting function

#Input:
#Datafrmae of interest - pandas.dataframe
# outlier_treatment = 0 if outliers are not removed. 
# Outlier treatment = 1 if outliers are remved

def generate_plot_inputs_carrWustaticHedge_PnL_regression(df_hedge, no_of_plots, \
                                                    prod_type, stock_ident, prod_moneyness, \
                                                    outlier_treatment = 0):
    if (outlier_treatment == 1):
        df_hedge['Outlier_diff'] = df_hedge['Pfl PnL - Realised'] - df_hedge['Pfl PnL - ConstVol'] 
        
        perctls_dyn = [np.percentile(np.array(df_hedge['Outlier_diff']), 2), \
                       np.percentile(np.array(df_hedge['Outlier_diff']), 98)]
        df_hedge = df_hedge[df_hedge['Outlier_diff'] > perctls_dyn[0]]
        df_hedge = df_hedge[df_hedge['Outlier_diff'] < perctls_dyn[1]]
    
    
    realised_pnL = df_hedge['Pfl PnL - Realised']
    model_pnLs = ['Pfl PnL - ConstVol', 'Pfl PnL - CarrWu']

    x_list = [np.array(realised_pnL).reshape(-1) for plot in range(0, no_of_plots)]
    y_list = [[np.array(df_hedge[model_pnLs[plot]]).reshape(-1)] for plot in range(0, no_of_plots)]
    
    #Titles
    if (prod_type == "CE"):
        option_type = "Call Option"
    else:
        option_type = "Put Option"
        
    figure_title_str = stock_ident + " - Realised PnL vs Static Hedge PnL - " + prod_moneyness + " " + option_type
    
    #Sub-title names
    subplot_title_str_list = ['Lasso Static Hedge', 'Carr-Wu Static Hedge']
    #Legends
    legends_str = [[subplot_title_str_list[plot], "Regression Fit"] for plot in range(0, no_of_plots)]
    
    #Axes titles
    xaxis_title_str = ["Realised PnL" for plot in range(0, no_of_plots)]
    yaxis_title_str = ["Model  PnL" for plot in range(0, no_of_plots)]
    
    for i in range(0, no_of_plots):
        if (i==1):
            yaxis_title_str[i] = ""
        else:
            None
    
    #Limits
    x_limits = [[np.amin(x_list[plot]), np.amax(x_list[plot])] for plot in range(0,no_of_plots)]
    y_limits = [[np.amin(y_list[plot]), np.amax(y_list[plot])] for plot in range(0,no_of_plots)]
    
    
    # Annotations
    lin_reg_fits = [LinearRegression(fit_intercept=False).fit(np.array(x_list[plot]).reshape(-1,1), \
                                                             np.array(y_list[plot][0]).reshape(-1,1)) for plot in range(0, no_of_plots) ] 
    
    other_annotations = ["R-Square: " + str(round(lin_reg_fits[plot].score(np.array(x_list[plot]).reshape(-1,1), \
                                                  np.array(y_list[plot][0]).reshape(-1,1))*100,2)) + "%" \
                          + "<br>" \
                          "Model PnL = " + str(round(lin_reg_fits[plot].coef_[0][0], 2)) + " * Realised PnL" \
                         for plot in range(0, no_of_plots) ]
    
    regression_coefficients = [lin_reg_fits[plot].coef_[0][0] for plot in range(0, no_of_plots)]
    y_reg_line = [np.array([regression_coefficients[plot] * x_list[plot]]).reshape(-1) for plot in range(0, no_of_plots)]
    y_list = [[y_list[plot][0], y_reg_line[plot]] for plot in range(0, no_of_plots)] 

    return figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, \
           legends_str, other_annotations, x_limits, y_limits, x_list, y_list


def generate_plots_carrWu(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, \
                  figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, \
                  other_annotations):
    
    fig = make_subplots(rows=no_of_rows, cols=no_of_cols, subplot_titles=subplot_title_str_list)
    fig.update_layout(height=400, width=900, \
                     title=dict(text=figure_title_str, y=0.95, x=0.5, xanchor='center', yanchor='top'), \
                     font=dict(color="black", size=14), showlegend=False)
    figure_positions = list(product(range(1, no_of_rows+1), range(1, no_of_cols+1)))
    colours_list = ['rgb(57,105,172)', 'rgb(207, 28, 144)', 'rgb(17, 165, 121)', 'rgb(230, 131, 16)', 'rgb(139, 0, 0)', 'rgb(0, 0, 139)', 'rgb(0, 139, 0)']
    legend_show_list = [False] + [False for i in range(0, no_of_plots-1)]
    
    for i in range(0, no_of_plots):
        for j in range(0, no_of_yvars):
            if (j==0):
                plt_color = colours_list[i]
                plt_mode='markers'
            else:
                plt_color = 'rgb(153, 153, 153)'
                plt_mode='lines'
            
            fig.add_trace(go.Scatter(x=list(x_list[i]), y=list(y_list[i][j]), \
                                     mode=plt_mode, marker=dict(size=4, color=plt_color), \
                                     name=legends_str[i][j], showlegend=legend_show_list[i]), \
                          row=figure_positions[i][0], col=figure_positions[i][1])
    
    for i in range(0, no_of_plots):
        fig.update_xaxes(title_text=xaxis_title_str[i], showgrid=True, range=x_limits[i], \
                        row=figure_positions[i][0], col=figure_positions[i][1])
        fig.update_yaxes(title_text=yaxis_title_str[i], showgrid=True, range=y_limits[i], \
                        row=figure_positions[i][0], col=figure_positions[i][1])
    
    annotations=[]
    annotations_extend = [dict(x=0.01, y=0.999, xref="paper", yref="paper", \
                              text=other_annotations[0], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                         dict(x=0.56, y=0.999, xref="paper", yref="paper", \
                              text=other_annotations[1], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=9)), \
                         ]
    annotations = [annotation for annotation in fig["layout"]["annotations"]]
    annotations = annotations + annotations_extend
    fig["layout"]["annotations"] += tuple(annotations)
    
    return fig



In [52]:
no_of_plots=2
no_of_rows = 1
no_of_cols = 2
no_of_yvars = 2
outlier_treatment = 0
figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, \
legends_str, other_annotations, x_limits, y_limits, x_list, y_list = generate_plot_inputs_carrWustaticHedge_PnL_regression(df_hedge_carrWu, no_of_plots, \
                                                    prod_type, stock_ident, prod_moneyness, \
                                                    outlier_treatment)

reg_carrWustatic_hedge_plot = generate_plots_carrWu(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, \
                          figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, \
                          other_annotations)
reg_carrWustatic_hedge_plot.show()

# if (outlier_treatment == 0):
#     reg_static_hedge_plot.write_image(output_data_path + "P2_"+ stock_ident + "_" + prod_moneyness + "_" + prod_type  \
#                          +'_static_regression_fit_carrWu.png')
# else:
#     reg_static_hedge_plot.write_image(output_data_path + "P2_"+ stock_ident + "_" + prod_moneyness + "_" + prod_type  \
#                          +'_static_regression_fit_carrWu_outlier_treated.png')    




In [28]:
# PnL Error Plots

def generate_plot_inputs_carrWuPnL_Errors(df_hedge, no_of_plots, \
                                                    prod_type, stock_ident, prod_moneyness, \
                                                    outlier_treatment = 0):

    model_pnLs = ['Pfl PnL - ConstVol', 'Pfl PnL - CarrWu']
    
#     df_hedge['PnL Diff: Dynamic Hedge'] = df_hedge['Pfl PnL - Realised'] - df_hedge['Pfl PnL - Dynamic']
    df_hedge['PnL Diff: Constant Vol Static Hedge'] = df_hedge['Pfl PnL - Realised'] - df_hedge['Pfl PnL - ConstVol]
    df_hedge['PnL Diff: CarrWu Static Hedge'] = df_hedge['Pfl PnL - Realised'] - df_hedge['Pfl PnL - CarrWu']    
    
    if (outlier_treatment == 1):
#         dyn_abs_error_pctl = np.percentile(np.absolute(np.array(df_hedge['PnL Diff: Dynamic Hedge'])),99)
        consVol_abs_error_pctl = np.percentile(np.absolute(np.array(df_hedge['PnL Diff: Constant Vol Static Hedge'])),99)
        carrWu_abs_error_pctl = np.percentile(np.absolute(np.array(df_hedge['PnL Diff: CarrWu Static Hedge'])),99)
        abs_pctl = np.amin(np.array([consVol_abs_error_pctl, \
                                     carrWu_abs_error_pctl]))
        
#         df_hedge['PnL Abs Diff: Dynamic Hedge'] = np.absolute(np.array(df_hedge['PnL Diff: Dynamic Hedge']))
        df_hedge['PnL Abs Diff: Constant Vol Static Hedge'] = np.absolute(np.array(df_hedge['PnL Diff: Constant Vol Static Hedge']))
        df_hedge['PnL Abs Diff: CarrWu Static Hedge'] = np.absolute(np.array(df_hedge['PnL Diff: CarrWu Static Hedge']))
        
#         df_hedge = df_hedge[df_hedge['PnL Abs Diff: Dynamic Hedge'] < abs_pctl]
        df_hedge = df_hedge[df_hedge['PnL Abs Diff: Constant Vol Static Hedge'] < abs_pctl]
        df_hedge = df_hedge[df_hedge['PnL Abs Diff: CarrWu Static Hedge'] < abs_pctl]
               
    pnL_diffs_names = ['PnL Diff: Constant Vol Static Hedge', 'PnL Diff: CarrWu Static Hedge']
    
    
    date_list = np.array(df_hedge['Date']).reshape(-1)
    get_date = lambda each_date: datetime.strptime(each_date, "%d-%b-%Y").date()
    get_date_vector = np.vectorize(get_date)
    
    date_list = get_date_vector(date_list)
    
    x_list = [date_list for plot in range(0, no_of_plots)]
#     y_list = [[np.absolute(np.array(df_hedge[pnL_diffs_names[0]])).reshape(-1), \
#               np.absolute(np.array(df_hedge[pnL_diffs_names[plot]])).reshape(-1)] for plot in range(1, no_of_plots+1)]
    y_list = [[np.array(df_hedge[pnL_diffs_names[0]]).reshape(-1), \
              np.array(df_hedge[pnL_diffs_names[plot]]).reshape(-1)] for plot in range(1, no_of_plots+1)]

    #Titles
    if (prod_type == "CE"):
        option_type = "Call Option"
    else:
        option_type = "Put Option"
        
    figure_title_str = stock_ident + ": PnL Errors - Lasso vs CarrWu Static Hedge - " + prod_moneyness + " " + option_type
    
    #Sub-title names
    subplot_title_str_list = ['']
    #Legends
    legends_str = [["Lasso Static Hedge", "CarrWu Static Hedge"] for plot in range(0, no_of_plots)]
    
    for i in range(0, no_of_plots):
        if (i!=0):
            legends_str[i][0] = ""
    
    #Axes titles
    xaxis_title_str = ["Date" for plot in range(0, no_of_plots)]
    yaxis_title_str = ["PnL Abs Difference" for plot in range(0, no_of_plots)]
    
    
    #Limits
    x_limits = [[np.amin(x_list[plot]) - timedelta(days=7), np.amax(x_list[plot]) + timedelta(days=7)] for plot in range(0,no_of_plots)]
    
    y_limits = [[np.amin(y_list[plot]) -5, np.amax(y_list[plot]) + 10] for plot in range(0,no_of_plots)]
    
        
    other_annotations = ["Mean Abs diff (Lasso): " + str(round(np.mean(np.absolute(y_list[plot][0])), 2)) \
                          + "<br>" \
                          "Mean Abs diff (CarrWu) = " + str(round(np.mean(np.absolute(y_list[plot][1])), 2)) \
                         for plot in range(0, no_of_plots)]
    return figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, \
           legends_str, other_annotations, x_limits, y_limits, x_list, y_list


def generate_carrWupnl_error_plots(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, \
                  figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, \
                  other_annotations):
    
    fig = make_subplots(rows=no_of_rows, cols=no_of_cols, subplot_titles=subplot_title_str_list)
    fig.update_layout(height=400, width=800, \
                     title=dict(text=figure_title_str, y=0.95, x=0.42, xanchor='center', yanchor='top'), \
                     font=dict(color="black", size=12), showlegend=True)
    figure_positions = list(product(range(1, no_of_rows+1), range(1, no_of_cols+1)))
    colours_list = ['rgb(57,105,172)', 'rgb(217,95,2)', 'rgb(207, 28, 144)', 'rgb(17, 165, 121)', 'rgb(230, 131, 16)', 'rgb(57,105,172)', \
                    'rgb(93,105,177)', 'rgb(57,105,172)', 'rgb(139, 0, 0)', 'rgb(0, 0, 139)', 'rgb(0, 139, 0)']
#     legend_show_list = [True] + [True for i in range(0, no_of_plots-1)]
    
    for i in range(0, no_of_plots):
        for j in range(0, no_of_yvars):
            if (j==1):
                plt_color = colours_list[0]
                plt_mode='lines+markers'
                legend_show=True
            else:
                plt_color = colours_list[8]
                plt_mode='lines+markers'
                if(i==0):
                    legend_show=True
                else:
                    legend_show=False
                      
            fig.add_trace(go.Scatter(x=list(x_list[i]), y=list(y_list[i][j]), \
                                     mode=plt_mode, marker=dict(size=4, color=plt_color), \
                                     name=legends_str[i][j], showlegend=legend_show), \
                          row=figure_positions[i][0], col=figure_positions[i][1])
    
    for i in range(0, no_of_plots):
        fig.update_xaxes(title_text=xaxis_title_str[i], showgrid=True, range=x_limits[i], \
                        row=figure_positions[i][0], col=figure_positions[i][1])
        fig.update_yaxes(title_text=yaxis_title_str[i], showgrid=True, range=y_limits[i], \
                        row=figure_positions[i][0], col=figure_positions[i][1])
    
    annotations=[]
    annotations_extend = [dict(x=0.01, y=0.999, xref="paper", yref="paper", \
                              text=other_annotations[0], showarrow=False, \
                              xanchor="left", yanchor="top", font=dict(color="black", size=11)), \
                         ]
    annotations = [annotation for annotation in fig["layout"]["annotations"]]
    annotations = annotations + annotations_extend
    fig["layout"]["annotations"] += tuple(annotations)
    
    return fig


# Regression plots for Static Hedging and Dynamic Hedging

no_of_plots=1
no_of_rows = 1
no_of_cols = 1
no_of_yvars = 2
outlier_treatment = 1
figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, \
legends_str, other_annotations, x_limits, y_limits, x_list, y_list = generate_plot_inputs_carrWuPnL_Errors(df_hedge, no_of_plots, \
                                                    prod_type, stock_ident, prod_moneyness, \
                                                    outlier_treatment)

reg_static_hedge_pnLerror_plot = generate_carrWupnl_error_plots(no_of_plots, no_of_rows, no_of_cols, no_of_yvars, \
                          figure_title_str, subplot_title_str_list, xaxis_title_str, yaxis_title_str, legends_str, \
                          other_annotations)
reg_static_hedge_pnLerror_plot.show()

# if (outlier_treatment == 0):
#     reg_static_hedge_pnLerror_plot.write_image(output_data_path + "P2_"+ stock_ident + "_" + prod_moneyness + "_" + prod_type  \
#                          +'_carrWu_lasso_pnL_error.png')
# else:
#     reg_static_hedge_pnLerror_plot.write_image(output_data_path + "P2_"+ stock_ident + "_" + prod_moneyness + "_" + prod_type  \
#                          +'_carrWu_lasso_pnL_error.png')   



In [45]:
# fig.write_image(output_data_path + "P1_Fig_Implied_VolSurface_OctNOv2019.png")

In [178]:
# print(px.colors.qualitative.Plotly)

In [179]:
# fig = px.colors.qualitative.swatches()
# fig.show()

In [1]:
# colours_plot = px.colors.qualitative.G10
# colours_plot + ['rgb(166,118,29)', 'rgb(124,124,124)']

# 'rgb(57,105,172)', 'rgb(207, 28, 144)', 'rgb(17, 165, 121)', 'rgb(230, 131, 16)'
# 'rgb(231, 138,195)'

In [2]:
# print(px.colors.qualitative.Dark2)

In [180]:
# print(list(product(range(1, 2+1), range(1, 2+1))))